In [1]:
from deepbrain import Extractor
import nibabel
import numpy as np
import pydicom
import os

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
import matplotlib.pyplot as plt

def convertNsave(arr, file_path, index=0):
    """
    `arr`: parameter will take a numpy array that represents only one slice.
    `file_path`: parameter will take the path to save the slices
    `index`: parameter will represent the index of the slice, so this parameter will be used to put 
    the name of each slice while using a for loop to convert all the slices
    """
    
    dicom_file = pydicom.dcmread('dcmimage.dcm')
    dicom_file.Rows = arr.shape[0]
    dicom_file.Columns = arr.shape[1]
    dicom_file.PhotometricInterpretation = "MONOCHROME2"
    dicom_file.SamplesPerPixel = 1
    dicom_file.BitsStored = 32
    dicom_file.BitsAllocated = 32
    dicom_file.HighBit = 31
    dicom_file.PixelRepresentation = 0
    dicom_file.PixelData = arr.astype('uint32').tobytes()
    dicom_file.save_as(file_path + f'_{index}.dcm')

def nifti2dicom_1file(nifti_dir, out_path, out_name):
    """
    This function is to convert only one nifti file into dicom series
    `nifti_dir`: the path to the one nifti file
    `out_path`: the path to output
    """

    ext = Extractor()
    nifti_file = nibabel.load(nifti_dir)
    nifti_array = nifti_file.get_fdata()
    nifti_array = nifti_array * (2**32 - 1) / np.max(nifti_array)
    mask = ext.run(nifti_array) > 0.5

    for slice_ in range(nifti_array.shape[0]):
        data_slice = nifti_array[slice_, :, :]
        mask_slice = mask[slice_, :, :]

        if 0.1 > (mask_slice.sum() / np.size(mask_slice)):
            continue

        convertNsave(data_slice, os.path.join(out_path, 'coronal', out_name), slice_)

    for slice_ in range(nifti_array.shape[1]):
        data_slice = nifti_array[:, slice_, :]
        mask_slice = mask[:, slice_, :]

        if 0.1 > (mask_slice.sum() / np.size(mask_slice)):
            continue

        convertNsave(data_slice, os.path.join(out_path, 'axial', out_name), slice_)

    for slice_ in range(nifti_array.shape[2]):
        data_slice = nifti_array[:, :, slice_]
        mask_slice = mask[:, :, slice_]

        if 0.1 > (mask_slice.sum() / np.size(mask_slice)):
            continue

        convertNsave(data_slice, os.path.join(out_path, 'sagittal', out_name), slice_)


def nifti2dicom_mfiles(nifti_dir, out_dir=''):
    """
    This function is to convert multiple nifti files into dicom files
    `nifti_dir`: You enter the global path to all of the nifti files here.
    `out_dir`: Put the path to where you want to save all the dicoms here.
    PS: Each nifti file's folders will be created automatically, so you do not need to create an empty folder for each patient.
    """

    files = os.listdir(nifti_dir)
    os.makedirs(os.path.join(out_dir, 'axial'))
    os.makedirs(os.path.join(out_dir, 'coronal'))
    os.makedirs(os.path.join(out_dir, 'sagittal'))
    for file in files:
        in_path = os.path.join(nifti_dir, file)
        nifti2dicom_1file(in_path, out_dir, file.split('.')[0])


In [3]:
nifti2dicom_mfiles('dataset/IXI-T1', 'slices_dicom')

Instructions for updating:
Use tf.gfile.GFile.
